In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
import pickle

In [ ]:
# 웹 드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
driver.get("https://www.siksinhot.com/search?keywords=%EC%84%9C%EC%9A%B8")

In [ ]:
search_list = [
    "강동구","강서구","강남구","도봉구","양천구","마포구","서대문구","은평구","서초구","구로구","중구",
    "동대문구","중랑구","광진구","송파구","강북구","관악구"
]

In [ ]:
driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div").click()

In [ ]:
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys("강북구")
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys(Keys.RETURN)

In [ ]:
all_pages = []
for search in search_list:
    input_box = driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div")
    input_box.click()
    time.sleep(4)
    
    search_input = driver.find_element(By.CSS_SELECTOR,"#searchInput")
    search_input.send_keys(search)
    search_input.send_keys(Keys.RETURN)
    time.sleep(4)
    try:
        while True:
            more_box = driver.find_element(By.CSS_SELECTOR, "#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > a")

            more_box.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except:
        page_source = driver.page_source
        all_pages.append(page_source)
        pass 
    
    

In [ ]:
len(all_pages)

In [ ]:
for page in all_pages:
    soup = bs(page,"lxml")
    soup.select()

In [5]:
url = "https://www.siksinhot.com/search?keywords=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC"
r = requests.get(url)
soup = bs(r.text,"lxml")


In [6]:
main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")

In [7]:
page_list = []

for page in all_pages:
    soup = bs(page,"lxml")
    main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")
    for i in main:
        dict_list = {}
        title = i.select_one("h2").text
        location = i.select_one(".cate>a:first-child").text
        division = i.select_one(".cate>a:last-child").text
        view = i.select_one(".counting").text
        score = i.select_one(".score").text
        book_mark = i.select_one(".counting>span:nth-of-type(2)").text
        url = i.select_one("figure > a").get('href')
        address = i.select_one("figure > a > img").get('alt').split(",")[-1] if i.select_one("figure > a > img") is not None else "None"
        
    
        dict_list["제목"] = title 
        dict_list["지역"] = location
        dict_list["분류"] = division
        dict_list["조회수"] = view
        dict_list["평점"] = score
        dict_list["북마크"] = book_mark
        dict_list["url"] = url
        dict_list["주소"] = address
        
        page_list.append(dict_list)

NameError: name 'all_pages' is not defined

In [8]:
page_list

[]

In [ ]:
data = pd.DataFrame(page_list)

In [9]:
data.to_csv("식신.csv",encoding="utf-8",index=False)

NameError: name 'data' is not defined

In [11]:
data = pd.read_csv("식신.csv")

In [12]:
data[data["제목"].str.contains("꼼박사")]

,제목,지역,분류,조회수,평점,북마크,url,주소
1027,꼼박사 건대자양본점,건대/자양/화양,해물탕/해물요리,40892,3.0,2.0,https://www.siksinhot.com/P/364407,서울특별시 광진구 아차산로 354


In [12]:
len(data)

2028

In [13]:
detail_pages = [] 
for links in data["url"][1028:]:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
    driver.get(links)
    time.sleep(3)
    
    try:
        while True:
            more_box2 = driver.find_element(By.CSS_SELECTOR, "#storeView > div.container > div.store_review > a")

            more_box2.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except: 
        page_source2 = driver.page_source
        detail_pages.append(page_source2)
        driver.quit()
    

In [16]:
len(detail_pages)

1000

In [4]:
with open("식신디테일정보.pkl","rb") as read_file:
    main_pages = pickle.load(read_file)

In [19]:
len(main_pages)

2028

In [18]:
main_pages.extend(detail_pages)

In [9]:
main_pages[-1]

'<html><head>\n        <title>꼼박사 건대자양본점 - 서울 강북, 건대 자양 화양 | 맛집검색 식신</title>\n        <meta data-react-helmet="true" charset="utf-8"><meta data-react-helmet="true" http-equiv="Content-Type" content="text/html; charset=utf-8"><meta data-react-helmet="true" name="title" content="꼼박사 건대자양본점 - 서울 강북, 건대 자양 화양 | 맛집검색 식신"><meta data-react-helmet="true" name="description" content="꼼박사 건대자양본점입니다. 꼬들꼬들 맛있는 꼼장어입니다. 서울 강북, 건대 자양 화양에 위치하고 있습니다. 영업시간은 매일 18:00 ~ 02:00입니다. 대표메뉴는 꼼장어 양념소금(2인분), 아나고입니다."><meta data-react-helmet="true" name="keywords" content="맛집, 서울 강북, 건대 자양 화양맛집, 맛집, 맛집, 꼼장어 양념소금(2인분)맛집, 씨푸드맛집, 꼼박사 건대자양본점"><meta data-react-helmet="true" name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi"><meta data-react-helmet="true" name="robots" content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1 "><meta data-react-helmet="true" name="format-detection" content="telep

In [20]:
# 이거 진짜 신중하게 써라ㅠㅠㅠㅠㅠ덮어지뮤ㅠㅠㅠㅠㅠㅠ으어 내 이틀
with open("메인디테일정보.pkl","wb") as file:
    pickle.dump(main_pages,file)

In [21]:
len(main_pages)

2028

In [2]:
with open("식신디테일정보2.pkl","rb") as read_file:
    detail_pages = pickle.load(read_file)

In [8]:
len(all_pages)

NameError: name 'all_pages' is not defined

In [10]:
all_pages[-1]

'<html><head>\n        <title>서초면옥 서초본점 - 서울 강남, 교대 서초 남부터미널 | 맛집검색 식신</title>\n        <meta data-react-helmet="true" charset="utf-8"><meta data-react-helmet="true" http-equiv="Content-Type" content="text/html; charset=utf-8"><meta data-react-helmet="true" name="title" content="서초면옥 서초본점 - 서울 강남, 교대 서초 남부터미널 | 맛집검색 식신"><meta data-react-helmet="true" name="description" content="서초면옥 서초본점입니다. 갈비찜과 함흥냉면이 인기있는 식당입니다. 서울 강남, 교대 서초 남부터미널에 위치하고 있습니다. 영업시간은 매일 11:00 ~ 21:30입니다. 대표메뉴는 회냉면, 물냉면입니다."><meta data-react-helmet="true" name="keywords" content="맛집, 서울 강남, 교대 서초 남부터미널맛집, 맛집, 맛집, 회냉면맛집, 한식맛집, 서초면옥 서초본점"><meta data-react-helmet="true" name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi"><meta data-react-helmet="true" name="robots" content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1 "><meta data-react-helmet="true" name="format-detection" content="telephone=no

In [ ]:
for k in bs(all_pages[-1],"lxml").select(".info_text"):
    print(k.text)

In [ ]:
for i in all_pages[1]:
    soup = bs(i,"lxml")
    for j in soup.select(".info_text"):
        print(j.select_one(".inline-div"))
    

In [10]:
all_pages.extend(detail_pages)

NameError: name 'all_pages' is not defined

In [ ]:
len(all_pages)

In [4]:
driver.find_element(By.CSS_SELECTOR,"#storeView > div.container > div.store_review > a").click()

NameError: name 'driver' is not defined